In [1]:
import torch
import numpy as np

from datasets.dataset import transform_dataset, kfold_dataset
from DRNet import train, DRNet
import time
import random

In [2]:
# Read datasets
name = 'nursery'
numclass=5 #类别数量
X, Y, X_headers, Y_headers = transform_dataset(name, method='onehot', negations=False, labels='ordinal')  #drnet的数据处理
print(max(Y),min(Y))
random.seed(time.time())


datasets = kfold_dataset(X, Y, shuffle=1)#k-折
X_train, X_test, Y_train, Y_test = datasets[0]



X_train_array=X_train.to_numpy()
Y_train_array=Y_train
X_train_mul=[]

#下循环就是把训练集按类别分开
# X_train_mul[i]装了类别为i的训练集
for i in range(0,numclass):
    tempta=X_train_array[np.where(Y_train_array==i)]
    X_train_mul.append(tempta)



4 0


C:\Users\Admin\anaconda3\envs\DR-net\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [3]:
dic={}

#这里采用一对一（OVO）
#枚举类别i j（i！=j）
for i in range(0,numclass):
    for j in range(i+1,numclass):
        #对类别为i和j的训练集进行合体（？）变成训练集（i，j）做标签 i为0类，j为1类
        X_train_cur=np.append(X_train_mul[i],X_train_mul[j],axis=0)
        Y_train_cur=[0]*len(X_train_mul[i])
        Y_train_cur.extend([1]*len(X_train_mul[j]))
        
        train_set = torch.utils.data.TensorDataset(torch.Tensor(X_train_cur), torch.Tensor(Y_train_cur))
        test_set = torch.utils.data.TensorDataset(torch.Tensor(X_test.to_numpy()), torch.Tensor(Y_test))
        #扔进去训练，没调参
        net = DRNet(train_set[:][0].size(1), 50, 1)
        train(net, train_set, test_set=test_set, device='cuda', lr=1e-2, epochs=2000, batch_size=400,
      and_lam=1e-2, or_lam=1e-5, num_alter=500)
        #训练完记录再字典里
        dic[(i,j)]=net


        





Epoch: 100%|██████████| 2000/2000 [03:21<00:00,  9.92it/s, loss=0.253, epoch accu=0.96, test accu=0.662, num rules=22, 
Epoch: 100%|██████████| 2000/2000 [01:43<00:00, 19.32it/s, loss=0.474, epoch accu=0.925, test accu=0.333, num rules=0, 
Epoch: 100%|██████████| 2000/2000 [03:15<00:00, 10.24it/s, loss=0.246, epoch accu=0.956, test accu=0.662, num rules=25,
Epoch: 100%|██████████| 2000/2000 [01:53<00:00, 17.61it/s, loss=0.441, epoch accu=0.968, test accu=0.447, num rules=48,
Epoch: 100%|██████████| 2000/2000 [01:43<00:00, 19.36it/s, loss=0.474, epoch accu=0.937, test accu=0.333, num rules=0, 
Epoch: 100%|██████████| 2000/2000 [03:17<00:00, 10.14it/s, loss=0.362, epoch accu=0.877, test accu=0.212, num rules=44,
Epoch: 100%|██████████| 2000/2000 [01:53<00:00, 17.56it/s, loss=0.497, epoch accu=0.936, test accu=0.339, num rules=1, 
Epoch: 100%|██████████| 2000/2000 [01:37<00:00, 20.54it/s, loss=0.00206, epoch accu=0.988, test accu=0.329, num rules=5
Epoch: 100%|██████████| 2000/2000 [00:06

In [4]:
#这个我瞎写的，看规则的好像，记不太清
dir='D:/document/tyf/tyf/VS/VS/vswork/baseline/rules/drnet/'
dir=dir+name+'.txt'
aver=0
with open(dir,'w') as f:
    num=0
    for i in range(0,numclass):
        for j in range(i+1,numclass):
            rs=dic[(i,j)].get_rules(X_headers)
            num+=len(rs)
            for r in rs:
                aver+=len(r)
                f.write(str(r)+'\n')
    print(num)
    aver=aver/num
    print(aver)
            


259
8.617760617760618


In [5]:
#投票
vote=np.zeros([len(X_test),numclass])


for i in range(0,numclass):
    for j in range(i+1,numclass):
        p=dic[(i,j)].predict(np.array(X_test))
        p=dic[(i,j)].predict(np.array(X_test))
        ind=np.where(p==0)[0]
        for indi in ind:
            vote[indi][i]+=1
        ind=np.where(p==1)[0]
        for indi in ind:
            vote[indi][j]+=1

print(vote)

[[1. 3. 1. 2. 3.]
 [2. 4. 1. 3. 0.]
 [1. 4. 1. 3. 1.]
 ...
 [2. 3. 1. 4. 0.]
 [2. 3. 1. 4. 0.]
 [4. 2. 1. 3. 0.]]


In [6]:
pred=[]
auuc=0

for id in range(0,len(X_test)):
    tempta=np.where(vote[id]==max(vote[id]))[0]
    pred.append(tempta[0])
    if pred[id]==Y_test[id]:
        auuc+=1
print(auuc/len(X_test))

0.9178240740740741
